<a name='1'></a>
## 1 - Packages

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL
import cv2
import os
import matplotlib.pyplot as plt
from glob import glob
from tensorflow import keras
from IPython.display import clear_output as cls
from fr_utils import *
import win32com.client as wincl
from multiprocessing import Pool

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
PADDING = 50
ready_to_detect_identity = True
windows10_voice_interface = wincl.Dispatch("SAPI.SpVoice")

<a name='1'></a>
## For collect image dataset on webcam

In [3]:
# # # Create directories for each person
# person_name = "Boom"
# os.makedirs(f"dataset/{person_name}", exist_ok=True)

# # Initialize webcam
# cap = cv2.VideoCapture(0)
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# img_count = 0

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.1, 4)

#     for (x, y, w, h) in faces:
#         img_count += 1
#         face = frame[y:y + h, x:x + w]
#         cv2.imwrite(f"dataset/{person_name}/{person_name}_{img_count+400}.jpg", face)
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

#     cv2.imshow("Face Capture", frame)

    
#     if cv2.waitKey(1) & 0xFF == ord('q') or img_count >= 100:  # Collect 100 images
#         break

# cap.release()
# cv2.destroyAllWindows()

In [4]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define the image size (input size for FRmodel)
IMG_SIZE = (160, 160)

# Prepare lists to hold the images and labels
images = []
labels = []

# Dataset path
dataset_path = './dataset/'

# Loop through each person in the dataset
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    
    if os.path.isdir(person_path):
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            
            # Load image
            img = cv2.imread(img_path)
            
            if img is not None:
                # Resize to 160x160
                img = cv2.resize(img, IMG_SIZE)
                
                # Normalize pixel values
                img = img / 255.0
                
                # Append the image and its label
                images.append(img)
                labels.append(person)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode the labels (e.g., 'Auu', 'Phupha', 'Boom')
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

In [5]:
def triplet_loss(y_true, y_pred, alpha = 0.3):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [6]:
from inception_blocks_v2 import *

FRmodel = faceRecoModel(input_shape=(3, 160, 160))

In [7]:
# from keras.models import Model
# from keras.optimizers import Adam

# # # Compile the model
# FRmodel.compile(optimizer=Adam(learning_rate=0.0001), loss=triplet_loss, metrics = ['accuracy'])

# # # Train the model (this is a simplified example)
# FRmodel.fit(tf.transpose(images, perm=[0, 3, 1, 2]), labels_encoded, epochs=10, batch_size=32)

In [8]:
# FRmodel.save_weights('FRmodel.weights.h5')

In [9]:
# Load the model weights from a file
FRmodel.load_weights('FRmodel.weights.h5')

In [10]:
# FRmodel.evaluate(tf.transpose(images, perm=[0, 3, 1, 2]), labels_encoded)

In [11]:
FRmodel.predict(np.expand_dims(tf.transpose(images[0], perm=[2, 0, 1]), axis = 0)).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(1, 128)

In [12]:
# def img_to_encoding(image, model):
#     """
#     Converts an image to an encoding using the given model.

#     Arguments:
#     image -- a numpy array of the image
#     model -- Keras model used to encode the image

#     Returns:
#     encoding -- the encoded representation of the image
#     """
#     # If the image is a path, load it; otherwise, assume it's already a NumPy array
#     if isinstance(image, str):
#         img = tf.keras.preprocessing.image.load_img(image, target_size=(160, 160))
#         img = np.array(img) / 255.0
#     else:
#         img = np.around(image / 255.0, decimals=12)

#     x_train = np.expand_dims(img, axis=0)
#     x_train = np.transpose(x_train, (0, 3, 2, 1))
#     encoding = model.predict(x_train)
#     return encoding

def img_to_encoding(image, model):
    if isinstance(image, str):
        img = tf.keras.preprocessing.image.load_img(image, target_size=(160, 160))
        img = np.array(img) / 255.0
    else:
        img = np.around(image / 255.0, decimals=12)

    # Ensure the input shape is correct (160, 160, 3)
    if img.shape != (160, 160, 3):
        img = cv2.resize(img, (160, 160))  # Resize the image if needed

    x_train = np.expand_dims(img, axis=0)  # Add batch dimension
    x_train = np.transpose(x_train, (0, 3, 2, 1))
    encoding = model.predict(x_train)
    return encoding

In [13]:
def compare_encodings(encoding1: np.ndarray, encoding2: np.ndarray, threshold: float) -> tuple:
    
    """
    Compare two face encodings using Euclidean distance and a threshold.
    If the distance is less than the threshold, the encodings are considered a match.
    
    Parameters:
    encoding1 (np.ndarray): Face encoding 1
    encoding2 (np.ndarray): Face encoding 2
    threshold (float): Threshold for considering two encodings as a match
    
    Returns:
    tuple: A tuple with two elements - a binary flag indicating whether the encodings are a match (1 if match, 0 otherwise), 
           and the distance between the encodings
    """
    
    # Compute the Euclidean distance between the two encodings
    dist = np.linalg.norm(encoding1 - encoding2)
    
    # Check if the distance is less than the threshold
    if dist < threshold:
        
        # Return a match and the distance
        return 1, dist
    
    else:
        
        # Return no match and the distance
        return 0, dist

In [14]:
database = {}
database["Phupha"] = img_to_encoding("dataset/Phupha/Phupha_149.jpg", FRmodel)
database["Boom"] = img_to_encoding("dataset/Boom/Boom_422.jpg", FRmodel)
database["Auu"] = img_to_encoding("dataset/Auu/Auu_337.jpg", FRmodel)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 775ms/step


In [15]:
def who_is_it(image, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    encoding = img_to_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dist = np.linalg.norm(db_enc - encoding)

        print('distance for %s is %s' %(name, dist))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        return None
    else:
        return str(identity)

In [16]:
# who_is_it("dataset/Auu/Auu_300.jpg", database, FRmodel)
# who_is_it("dataset/Boom/Boom_500.jpg", database, FRmodel)
# who_is_it("dataset/Phupha/Phupha_200.jpg", database, FRmodel)

In [17]:
def webcam_face_recognizer(database):
    """
    Runs a loop that extracts images from the computer's webcam and determines whether or not
    it contains the face of a person in our database.

    If it contains a face, an audio message will be played welcoming the user.
    If not, the program will process the next frame from the webcam
    """
    global ready_to_detect_identity

    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(0)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame

        # We do not want to detect a new identity while the program is in the process of identifying another person
        if ready_to_detect_identity:
            img = process_frame(img, frame, face_cascade)   
        
        key = cv2.waitKey(100)
        cv2.imshow("preview", img)

        if key == 27: # exit on ESC
            break
    cv2.destroyWindow("preview")

In [18]:
def process_frame(img, frame, face_cascade):
    """
    Determine whether the current frame contains the faces of people from our database
    """
    global ready_to_detect_identity
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Check if faces are being detected
    if len(faces) == 0:
        print("No face detected")
    else:
        print(f"{len(faces)} face(s) detected")

    # Loop through all the faces detected and determine whether or not they are in the database
    identities = []
    for (x, y, w, h) in faces:
        x1 = x-PADDING
        y1 = y-PADDING
        x2 = x+w+PADDING
        y2 = y+h+PADDING

        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),2)

        identity = find_identity(frame, x1, y1, x2, y2)

        if identity is not None:
            print(f"Identity found: {identity}")
            identities.append(identity)
            # Put the identity text on the frame
            cv2.putText(img, identity, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        else:
            print("No identity found for the detected face")

    if identities:
        cv2.imwrite('example.png', img)
        ready_to_detect_identity = False
        pool = Pool(processes=1)
        # Process in a separate thread so the camera doesn't freeze
        pool.apply_async(welcome_users, [identities])
    return img

In [19]:
def find_identity(frame, x1, y1, x2, y2):
    """
    Determine whether the face contained within the bounding box exists in our database

    x1,y1_____________
    |                 |
    |                 |
    |_________________x2,y2

    """
    height, width, channels = frame.shape
    # The padding is necessary since the OpenCV face detector creates the bounding box around the face and not the head
    part_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]

    # Check if the part image is valid
    if part_image.size == 0:
        print("Invalid image region")
        return None

    # Get the identity
    identity = who_is_it(part_image, database, FRmodel)
    
    return identity

In [20]:
def welcome_users(identities):
    """ Outputs a welcome audio message to the users """
    global ready_to_detect_identity
    welcome_message = 'Welcome '

    if len(identities) == 1:
        welcome_message += '%s, have a nice day.' % identities[0]
    else:
        for identity_id in range(len(identities)-1):
            welcome_message += '%s, ' % identities[identity_id]
        welcome_message += 'and %s, ' % identities[-1]
        welcome_message += 'have a nice day!'

    windows10_voice_interface.Speak(welcome_message)

    # Allow the program to start detecting identities again
    ready_to_detect_identity = True

In [21]:
if __name__ == "__main__":
    webcam_face_recognizer(database)

No face detected
No face detected
1 face(s) detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step
distance for Phupha is 0.102593005
distance for Boom is 0.10027188
distance for Auu is 0.119707
Identity found: Boom
